In [ ]:
import os
import sys
import torch
import numpy as np

from Datasets.Datasets_Functions import *
from Visualization import Visualization

from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

In [ ]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
dataset_name = "r100"
dataset_properties = hashmap[dataset_name]
batch_size = dataset_properties['batch_size'] = 256
n_clusters = dataset_properties['n_clusters'] 
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)

In [ ]:
visualization = Visualization()

### IDec

In [ ]:
n_clusters = dataset_properties['n_clusters']

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size']

# Pre-Training Epochs and Learning Rate
n_pret_epochs = 100
pret_lr = 5e-4

# Lamdas, Training Epochs and Learning Rate
n_epochs = 100
lr = 5e-4
alpha = 1.0
gamma = 0.1
momentum = 0.9
latent_dim = 2
n_channels = 1
negative_slope = 0 
is_MLP_AE = True

if(is_MLP_AE):
    idec_autoencoder = IDECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim , negative_slope=negative_slope)
else:  
    input_dim = 1
    idec_autoencoder = IDECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope, n_channels=n_channels) 
idec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
idec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
idec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha, gamma=gamma)
idec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
idec_autoencoder.set_path()
idec_autoencoder = idec_autoencoder.to(device)

pretrain = True
if(pretrain):
    idec_autoencoder.pretrain_autoencoder()
    idec_autoencoder.save_pretrained_weights()
else:
    model_save_path = idec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    idec_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
idec_autoencoder.kmeans_initialization()
idec_autoencoder.train_autoencoder()
cluster_centers = idec_autoencoder.get_cluster_centers()
_, idec_autoencoder_reduced_data, labels = idec_autoencoder.get_latent_data()
General_Functions().save_excel(idec_autoencoder.data_dir_path, idec_autoencoder.df_eval)

In [ ]:
visualization.plot(idec_autoencoder_reduced_data, labels, labels, cluster_centers, idec_autoencoder.data_dir_path)